In [ ]:
#this can be used to join 2 catalogs for a joint fit. Scaling a column is also included. 

In [ ]:
import numpy as np
from astropy.table import Table, join, vstack
#open 2 files to join
file_matcha = ""
file_xmm = ""
catalog_matcha = Table.read(file_matcha)
catalog_xmm = Table.read(file_xmm)


print(f"The number of columns in `catalog_matcha` are {len(catalog_matcha.colnames)}")
print(f"The number of rows in catalog_matcha are {len(catalog_matcha)}.")
print(f"The number of rows are in catalog_xmm are {len(catalog_xmm)}.")

catalog_xmm[0:5]

In [ ]:
#column names must be identical to join the two catalogs with this method. Any "unmatched" columns will not be included in the final joint catalog. 
#make required cuts (instead of using CluStR to make cuts) for flags that are not present in both catalogs.
#An alternative method would be to add columns to the catalogs to have all matching flags, then run it through CluStR. 

#General flags used.

catalog_matcha2 = catalog_matcha[catalog_matcha['bad_mode'] == False]
#CluStR will automatically delete nans, but it can be useful to do before so you know how many clusters come from each catalog
catalog_matcha3 = catalog_matcha2[~np.isnan(catalog_matcha2["r2500_temperature_scaled"])]
catalog_matcha4 = catalog_matcha3[catalog_matcha3['edge_r2500'] == False]
catalog_matcha5 = catalog_matcha4[catalog_matcha4['edge_bkgd'] == False]
catalog_matcha6 = catalog_matcha5[catalog_matcha5['masked'] == False]
catalog_matcha7 = catalog_matcha6[catalog_matcha6['overlap_r500'] == False]
catalog_matcha8 = catalog_matcha7[catalog_matcha7['overlap_r2500'] == False]
catalog_matcha9 = catalog_matcha8[catalog_matcha8['overlap_bkgd'] == False]
catalog_matcha10 = catalog_matcha9[catalog_matcha9['500_kiloparsecs_SNR'] > 9.0]
catalog_matcha11 = catalog_matcha10[catalog_matcha10['Redshift'] > 0.2]
catalog_matcha12 = catalog_matcha11[catalog_matcha11['Redshift'] < 0.65]
#don't forget to delete repeat clusters. 
#Sometimes the catalogs don't have the same column name for MEM_MATCH_ID, so the column name in one of the catalogs needs to be renamed first to match.
#Edit to rename if needed: 
#catalog_xmm['MEM'].name = "MEM_MATCH_ID"
#make a column for repeats. 
catalog_matcha['repeat'] = np.isin(catalog_matcha['MEM_MATCH_ID'], catalog_xmm['MEM_MATCH_ID'])
#delete repeats
catalog_matcha13 = catalog_matcha12[catalog_matcha12['repeat'] == False]

#amount of clusters left after cuts. Make sure to use new catalog name that with the cut clusters. 
print(f"The number of columns are {len(catalog_matcha13.colnames)}")
print(f"The number of rows are {len(catalog_matcha13)}.")
catalog_matcha13[0:3]

#repeat this for second catalog if it needs any cuts

In [ ]:
#for when one of the catalogs needs something to be scaled, otherwise skip. 
#if using CluStR to complete the scaling relation, remember to set the pivot point to 1 and change where ln is used to log 10

#from the form: log10(Tx_chandra) = a*log10(Tx_xmm) + b. 
#some algebra:
#10^(log10(Tx_xmm) = 10^((log10(Tx_chandra) - b)/a)
#Tx_xmm = 10^((log10(Tx_Chandra) – b)/a))
#final relation:
#Tx_xmm = Tx Chandra Scaled = 10^((log10(TxChandra) – b)/a))... and here it is: 

#catalog_chandra["r2500_temperature_scaled"] = 10**(((np.log10(catalog_chandra["r2500_temperature"])-b)/a))

#there are alternative ways to do this scaling relation. xmm was chosen as the y value (even though its more algebra), to compare to past papers where xmm was scaled instead of chandra. 

In [ ]:
#choose a naming convention from one of the catalogs and rename all of the columns to match. All columns you want included in the final catalog need to have the same name in each catalog. 
catalog_xmm[0:5]
catalog_xmm.rename_column('kT','r2500_temperature')
catalog_xmm['Z_1'].name = "Redshift"
catalog_xmm['LAMBDA_CHISQ'].name = "lambda"
catalog_xmm['LAMBDA_CHISQ_E'].name = "lambda_err_low"
catalog_xmm['LAMBDA_CHISQ_E1'].name = "lambda_err_high"
catalog_xmm['kT_p'].name = "r2500_temperature_err_high_scaled"
catalog_xmm['kT_m'].name = "r2500_temperature_err_low_scaled"
catalog_xmm[0:5]

In [ ]:
#create an empty table. 

catalog_xmm_matcha = Table()

#use vstack to combine the catalogs. 
catalog_xmm_matcha = vstack([catalog_xmm, catalog_matcha13],join_type='inner')

print(f"The number of columns for `catalog_matcha_detected` are {len(catalog_xmm_matcha.colnames)}")
print(f"The number of rows are {len(catalog_xmm_matcha)}.")

#display table
catalog_xmm_matcha[0:20]

In [ ]:
#save catalog. 
#if doing csv instead, remember to change the format to "csc"
catalog_xmm_matcha.write("matcha_xmm_joint_scaled_temp_err_no_repeat_r2500_v2.fits", format="fits")